In [3]:
from IPython.display import Image

웨이모 데이터 셋은 구글 waymo에서 제공하는 자율주행 관련 open datasets입니다. 크게 perception, motion datasets으로 나누어져 있습니다. 이 중에서 인지하고 관련된 Perception Dataset를 살펴 보겠습니다

Perception Dataset는 1,950개의 세그먼트 데이터가 포함되어 있습니다. 각 세그먼트는 다양한 지역과 환경에서 센서당 10Hz(390,000 frames)로 20초동안 수집된 데이터입니다.

데이터는 Sensor data, Labeled data으로 구분됩니다. 
Sensor data는
- 라이다 데이터
- 카메라 데이터
- 센서 calibrations(보정) 데이터 (센서융합을 위한 데이터)
- 라이다에서 카메라  projections 데이터 
- 라이다 월드 좌표에서 이미지 좌표로 투영시킨 데이터(센서융합)

으로 이루어져 있습니다. 


Labeled data는 
차량, 보행자, 자전거 타는 사람, signs에 대한 labels
- lidar data (1,200 segments)의 labels, 
- camera data(1,000 segments)의 labels, 
- tracking IDs 가 포함된 3D bounding box labels,
- tracking IDs 가 포함된 2D bounding box labels

으로 구성되어 있습니다.


먼저 sensor data 중에서 Lidar data를 살펴보겠습니다. lidar datasets는 차 위에 mid-range lidar 한 개의 data와 차의 앞, 사이드 왼쪽 오른쪽, 뒤쪽에 부착된 short-range-lidars의 데이터 이렇게 총 5개의 lidar에 대한 data로 이루어져 있습니다.
사용된 라이더의 특징은
- mid-range-lidar : 최대 75m 범위
- short-range-lidar : 최대 20m 범위
- 모든 데이터에 대해 2개의 강한 intensity 값을 반환


그리고 포인트 클라우드의 특징은
- 범위 이미지로 포인트 클라우드에서 인코딩
- 각 lidar당 2개의 범위 이미지가 제공되고 2개의 가장 강력한 return 제공 
- 총 4개의 채널이 반환
  - channel 0 : 범위값
  - channel 1 : 라이다의 강도
  - channel 2 : 라이다의 elongation
  - channel 3 : is_in_nlz (1 or -1) : 1이면 in, -1이면 not in

입니다. 여기에서 lidar의 elongation은 nominal width보다 pulse가 더 길어지는 길이를 의미합니다. 즉 pulse가 굴절되어 나타나는 길이입니다. 
그리고 nlz는 레이블이 지정되지 않은 영역을 의미하는 “No Label Zone”의 약어입니다.


lidar data에서 기본적인 4채널외에도 추가로 lidar to camera projection을 위한 6채널을 제공하고 있습니다.
 
6개의 채널
- channel 0 : 카메라 이름
- channel 1 : x (axis along image width)
- channel 2 : y (axis along image height)
- channel 3 : 2nd projection의 camera name
- channel 4 : x (axis along image width)
- channel 5 : y (axis along image height)

이러한 데이터에 3D bounding box labels를 제공합니다. 이때 라이다의 labels는 고유 추적 ID와 함께 차량의 프레임 안에서 3D 7-DOF(degree of freedom)을 가진 bounding box입니다


그 다음 camera data를 살펴보겠습니다. camera datasets는 차 전방, 전방 왼쪽, 전방 오른쪽, 사이드 왼쪽, 사이드 오른쪽 총 5개의 camera에 대한 data로 이루어져 있습니다. 
카메라 data의 정보는

- JPEG 형식
- 각 쌍에 대한 하나의 camera images
- image bytes
- vehicle pose
- velocity (image center에 대한 노출시간에 해당)
- 롤링 셔터 타이밍 정보 (image center에 대한 노출시간에 해당) 

으로 구성되어 있습니다.  4,5,6 번은  lidar to camera projection을 할 때 유용한 값입니다.


이러한 데이터에 2D bounding box labels를 제공합니다. camera labels의 특징은
- tight-fitting(꽉 맞는)
- axis-aligned(축정렬)
- 글로벌한 고유 추적 ID
- bounding box가 개체의 보이는 부분만 포함
- 차량, 보행자, 자전거타는 사람에 대한 labels 제공
- 물체 추적 correspondences를 제공하지 않는다

입니다.


waymo datasets의 데이터를 살펴보겠습니다.

waymo datasets는 크게 
- context   
- camera_labels       
- image 
- laser_labels
- lasers
- no_label_zones
- pose
- projected_lidar_labels
- timestamp_micros
 으로 구성되어 있습니다. 

먼저 dataset의 context를 살펴보겠습니다. 이 context안에는camera lidar calibration에 필요한 내부 외부값 값과 beam에 대한 값이 있습니다.


### context

```python 
name: "11004685739714500220_2300_000_2320_000"
```
먼저 고유 ID가 정의됩니다.


``` python
camera_calibrations {
  name: FRONT
  intrinsic: 2055.556149361639

  extrinsic {
    transform: 0.9998926849887427
  }
  width: 1920
  height: 1280
  rolling_shutter_direction: LEFT_TO_RIGHT
}
```
camera calibrations 요소가 보입니다.우선 위 표의 값은 한눈에 볼수 있게 값의 갯수를 삭제시켰습니다. 이 값들은 카메라 이미지 보정에 필요한 값들입니다. 그 구성을 살펴보면
- name                      : 카메라 위치
- intrinsic                 : 카메라 내부 파라미터
- extrinsic                 : 카메라 외부 파라미터
  - transform             : 카메라좌표계와 월드좌표계 사이  평행이동변환값 
- width                     : 이미지 가로 픽셀수
- height                    : 이미지 세로 픽셀수
- rolling_shutter_direction : 개별 이미지프레임 씬을 주사하는 방식

입니다. 그리고 이러한 camera_calibrations 는 카메라마다 지정이 되어있어 (FRONT,FRONT_LEFT,FRONT_RIGHT,SIDE_LEFT,SIDE_RIGHT)총 5개의 요소가 존재합니다.


```python
laser_calibrations {
  name: FRONT
  beam_inclination_min: -1.5707963267948966
  beam_inclination_max: 0.5235987755982988
  extrinsic {
    transform: 0.9998652264631824

  }
}
```
laser_calibrations 요소를 확인할 수 있습니다. 우선 위 표의 값은 한눈에 볼수 있게 값의 갯수를 삭제시켰습니다. 이 값들은 lidar 보정에 필요한 값들입니다.그 구성을 살펴보면
- name                         : lidar 위치
- beam_inclination_min         : 라이다 내부 파라미터
- beam_inclination_max         : 라이다 내부 파라미터
- extrinsic                    : 라이다 외부 파라미터 
  - transform                  : radian각도값

1은 라이다의 위치 정보를 알려줍니다. 위치정보는 FRONT, REAR, SIDE_LEFT, SIDE_RIGHT, TOP 이렇게 총 5개입니다.그리고 2,3,4는 라이다 센서값을 매핑시키는데 사용됩니다.

```python
stats {
  laser_object_counts {
    type: TYPE_VEHICLE
    count: 13
  }
 laser_object_counts {
    type: TYPE_PEDESTRIAN
    count: 21
  }
 laser_object_counts {
    type: TYPE_SIGN
    count: 10
  }
time_of_day: "Day"
  location: "location_sf"
  weather: "sunny"
  camera_object_counts {
    type: TYPE_VEHICLE
    count: 9
  }
 camera_object_counts {
    type: TYPE_PEDESTRIAN
    count: 10
  }
```
stats라는 요소가 있습니다. 이는 프레임에 lidar , camera로 인식한 object가 어떤 종류이고 몇 개인지 그리고 날씨와 위치 정보를 포함하고 있습니다.


그리고 dataset의 camera label과 image를 살펴보겠습니다. 이 데이터는 이미지 데이터와 인식하고자 하는 object의 위치정보가 들어간 2D label data가 들어있습니다.

In [ ]:
![title](image/1.png)
위 사진은 waymo dataset의 tutorial에서 가져온 image값과 camera label값입니다. 자세히 살펴보면  사진에 빨간색 박스를 볼수가 있는데 그것이 label입니다

```python
box {
  center_x: 1875.489705
  center_y: 821.69859
  width: 262.10984999999994
  length: 89.02059000000008
}
type: TYPE_VEHICLE
id: "2b47a269-226d-4cde-b880-8f100246c1f5"
```
camer_labels.labels 에서 가져온 예시 데이터 입니다. 

box의 요소를 살펴보면
- center_x                    : label box의 center x pixel 위치값
- center_y                    : label box의 center y pixel 위치값
- width                          : label box의 가로 길이 pixel 값
- length                        : label box의 세로 길이 pixel 값

인 box의 위치정보가 들어 있습니다.
그리고 type에는 그 label이 된 객체의 정보와 id에는 고유 ID가 저장되어 있습니다.



In [ ]:
그리고 dataset의 lidar data를 살펴보겠습니다.
lidar의 정보에서 range_images를 추출해 보겠습니다.
먼저 range image는 다음과 같은 구성을 지닙니다.
- data                        :  범위 이미지 데이터
- name                      :  이미지 title
- layout                     :   plt layout
- vmin                       :   전달된 데이터의 최소값
- vmax                      :   전달된 데이터의 최대값
- cmap                      :   color map

![title](image/2.png)


In [ ]:
![title](image/3.png)

이러한 range_image와 frame 그리고 camera_projections, range_image_top_pose 데이터로 Point Cloud를 나타낼수 있습니다. point는 크게 points와 cp_points로 나뉘는데 points는 차 프레임의 3D 지점을 의미하고, cp_points는 각 점에서의 카메라 투영값입니다. 이러한 값들을 조합하여 point cloud를 나타내면 다음과 같습니다.

In [ ]:
![title](image/4.png)

그리고 이렇게 만들어진 map을 카메라에 투영시키면 다음과 같은 이미지가 형성됩니다.

## NUIMAGES


NUIMAGES는 NUSCENES에서 제공하는 자율주행 관련 open datasets입니다. 특징은

- 6초당 93k 동영상 클립(150시간 운전)
- 주석이 달린 94k 및 주석이 없는 110만 이미지
- 500개의 로그에서 대규모 이미지 데이터 세트 label을 지정
- 보스턴과 싱가포르의 위치 환경에서의 데이터
- 날씨 데이터 포함(눈 비 야간)
- 이미지의 75%는 희귀한 클래스에 중점(자전거)
- 이미지의 25%는 대표 데이터 세트 보강
- 2D 경계 상자 및 인스턴스 마스크가 있는 800K 주석이 달린 개체
- 백그라운드 클래스를 위한 10만개의 세분화 마스트
- 라이다, pose 같은 특성 데이터

입니다.


data format은 다음과 같습니다.
```python
attribute {
   "token"
   "name"
    "description"
}
```
주차/정지/이동중인 차량과 같은 속성 값입니다. token은 고유 식별자, name은 속성 이름입니다.

``` python
calibrated_sensor {
    "token"
     "sensor_token"  
    "translation" 
    "rotation"
     "camera_intrinsic"
    "camera_distortion”
}
```
카메라의 보정값에 대한 정보입니다. token은 고유 식별자, sensor_token은 센서 타입입니다.
translation, rotation, intrinsic, distortion 은 위 waymo에서 설명한 카메라 보정을 위한 값들입니다.


``` python
category {
   "token":                     
   "name":              
   "description": 
}
```
사람이나 차량을 분류해주는 category입니다.

```python
ego_pose {
    "token":
    "translation":    
    "rotation":      
    "timestamp":               
    "rotation_rate":             
    "acceleration":              
    "speed":                   
}
```
timestamp, 차량의 pose, 글로벌 좌표계 관련 데이터입니다. 즉 ego_pose는 라이더 맵 기반 localizaiton 알고리즘의 출력입니다.

```python 
log {
   "token": 
   "logfile":  .
   "vehicle":
   "date_captured":   // 년도, 월 ,날짜 데이터
   "location":             // 데이터 취득 위치
} 
```
데이터가 추출된 로그에 대한 정보입니다.

```python
object_ann {
    "token":       
   "sample_data_token":      
    "category_token":     .
    "attribute_tokens":    
    "bbox":             
    "mask":                
}
```
이미지의 주석입니다. 각 개체는 2D box, 2D 인스턴스 마스크, 카테고리별 attribute가 주석으로 표시됩니다.

```python
sample_data {
   "token
   "sample_token"
   "ego_pose_token":
   "calibrated_sensor_token"
   "filename":            
   "fileformat":    .
   "width":         
   "height":           
   "timestamp":   
   "is_key_frame":   
   "next":  
   "prev":.
}
```
데이터를 추출한 시기와 이미지에 대한 정보가 포함되어 있습니다. sample_data는 모든 이미지를 포함합니다. 모든 keyframe에 대해 2Hz 간격으로 과거 6, 미래 6개의 sweeps를 포함합니다. 이 값에서 timestamp값은 keyframe camera 샘플 데이터 time stamp에서 상
속됩니다.



```python
sample {
   "token":    
   "timestamp":        
   "log_token":         
   "key_camera_token":     
}
```
샘플은 주석이 달린 keyframe입니다. 모든 샘플에서는 최대 13개의 camera sample data가 있습니다.


``` python 
sensor {
   "token":    
   "channel":             
   "modality":         
}
```
sensor 에 대한 type 정보 입니다.

```python 
surface_ann {
   "token":                   
    "category_token":   
    "mask":    
}
```
이미지의 배경 개체의 주석입니다. 2d 분할 마스크 형태로 주석이 달려 있습니다.

이제 실제로 이미지를 꺼내서 확인해보겠습니다. NUSCENES 듀토리얼을 참고하였습니다.

![title](image/5.png)

데이터의 key값입니다.

![title](image/6.png)

이미지에 2D labeling이 표시된 사진입니다


![title](image/7.png)

2d 분할 마스크 형태로 나타낸 사진입니다.

![title](image/8.png)

실제 주석입니다.

## TuSimple

TuSimple의 lane_detection dataset은 자율주행 트럭 회사인 TuSimple에서 제공하는 차선 감지용 데이터셋입니다. 20프레임 1초 길이의 비디오 클립 7,000개 제공합니다.

데이터 세트는 
- training set : 3626개의 비디오 클립과, 3626개의 주석
- testing set : 2782개의 비디오 클립

으로 구성되어 있습니다.


비디오 클립은 1초에 20개의 프레임으로 이루어져 있으며 카메라의 시야 방향은 주행 방향과 비슷합니다. 

데이터 세트 취득시 환경은 날씨 좋은 기상조건이며 2차선 3차선 4차선 그 이상의 차선 데이터가 포함되어 있습니다.

주석의 유형은 차선 표시를 위한 폴리선 형태로 기입되어 있습니다.


```python
dataset
  |
  |----clips/                   # video clips
  |------|
  |------|----some_clip/        # Sequential images for the clip, 20 frames
  |------|----...
  |
  |----tasks.json  
```
훈련 .데이터 세트의 디렉토리 구조입니다. video clips와 json 파일이 있는데 json 파일은 clip 디렉토리 데이터를 사용하는 방법이 기술되어 있습니다.

```python
{
    'raw_file'
    'lanes'
    'h_samples'
}
```
line 데이터의 format입니다. 각각의 jaso 파일은 마지막 20번째 프레임의 클립에 대한 label data입니다.
각각의 요소를 살펴보면
- raw_file : video clip의 20번째 frame의 file 경로입니다.
- lanes :차선 list 값입니다.  각 요소값은 이미지의 weight 값입니다. 
- h_samples : len(h_samples) == len(lanes[i]) 를 의미하는 라인에 해당되는 높이 값 list입니다.


```python
{
  "lanes": [
        [-2, -2, -2, -2, 632, 625, 617, 609, 601, 594, 586, 578, 570, 563, 555, 547, 539, 532, 524, 516, 508, 501, 493, 485, 477, 469, 462, 454, 446, 438, 431, 423, 415, 407, 400, 392, 384, 376, 369, 361, 353, 345, 338, 330, 322, 314, 307, 299],
        [-2, -2, -2, -2, 719, 734, 748, 762, 777, 791, 805, 820, 834, 848, 863, 877, 891, 906, 920, 934, 949, 963, 978, 992, 1006, 1021, 1035, 1049, 1064, 1078, 1092, 1107, 1121, 1135, 1150, 1164, 1178, 1193, 1207, 1221, 1236, 1250, 1265, -2, -2, -2, -2, -2],
        [-2, -2, -2, -2, -2, 532, 503, 474, 445, 416, 387, 358, 329, 300, 271, 241, 212, 183, 154, 125, 96, 67, 38, 9, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2],
        [-2, -2, -2, 781, 822, 862, 903, 944, 984, 1025, 1066, 1107, 1147, 1188, 1229, 1269, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2]
       ],
  "h_samples": [240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710],
  "raw_file": "path_to_clip"
}
```
lane data의 예시입니다. 여기에서 -2 값의 의미는 차선 표시가 없다는 의미입니다.

```python
{
  'raw_file'
  'lanes'
  'run_time'
}
```
evaluation format입니다. raw_file, lanes는 line format과 동일합니다. run_time은 클립에서 각 프레임의 running time 입니다.

![title](image/9.png)

예시 자료 입니다.

## GM의 자회사인 Cruise의 오픈소스 Webviz

Webviz는 도로와 시뮬레이션에서 차량이 내리는 결정을 시각화하는 web기반 애플리케이션입니다. ros의 bag파일과 연동이 가능하여 ros를 통해 얻은 데이터를 시각화할 수 있습니다.


Webviz는 다음과 같은 도구를 가지고 있습니다..
- 1. webviz-core
- 2. regl-worldview
- 3. cruise-automation/hooks
- 4. cruise-automation/button
- 5. cruise-automation/tooltip


각각을 살펴보면 1번 webviz-core는 라이더, 카메라등의 센서 데이터가 ros를 통해 저장된 ros bag 파일을 검사해주는 도구입니다. 
2번 regl-worldview는 regl을 사용하여 2D, 3D 장면을 렌더링 하기 위한 React 라이브러리 입니다.
여기에서 regl이란 webgl(web 기반 그래픽 라이브러리)의 functional abstraction입니다. 그리고 React는 페이스북이 만든 사용자 UI구축을 위한 라이브러리입니다. 즉 웹기반 그래픽 라이브러리를 사용 편의성과 확장성이 좋은 UI로 만든 도구입니다. 
cruise-automation / hooks는 재사용 가능한 React hooks목록입니다. React hook는 class component, render을 하지 않고 function component에서 state를 가질 수 있게 되어 함수형 프로그래밍을 지원하는 도구입니다.


오픈소스 webviz를 사용하여 프로그래머는 라이더 센서를 보정하고 자동차가 실제 도로를 어떻게 인지하고 있는지, 그리고 어떻게 움직였는지를 확인할 수 있습니다.

In [ ]:
![title](image/10.png)

webviz의 tutorial입니다. 전체적인 창을 살펴보면 왼쪽에 라이더 point cloud를 3d 렌더링하고 있습니다. 이를 통해 라이더 센서를 보정할 수 있습니다. 그리고 오른쪽 상단을 보면 카메라 image값을 확인 할 수있습니다.

## 일본 나고야 대학의 Autoware 플랫폼

In [ ]:
autoware는 ROS 기반의 오픈소스 software입니다. autoware는 자율주행 자동차에 필수적인 localization, 장애물 탐지, path planning, path following 등의 기술을 제공합니다. 주로 라이다와 gps를 사용하여 localization을 하고 라이다와 GNSS(global navigation Satellite system)을 사용하여 객체를 감지 할 수 있습니다.
![title](image/11.png)

autoware는 위 그림처럼 크게 4가지 모듈로 구성되어 있습니다. 


- localization은 GPS와 라이다 센서를 퓨전하여 사용합니다. 먼저 GPS로부터 차량의 위도, 경도 좌표 값을 받아 옵니다. 그리고 이 좌표 데이터를 라이다로부터 얻은 3D point map과 매칭시켜 자차의 pose 데이터를 얻을 수 있습니다. 좌표 데이터와 3D point map을 매칭시킬 때 NDT메칭 알고리즘을 사용합니다. 이 알고리즘은 3D point의 좌표들을 메칭 확률이 최대치가 되도록 이동시킵니다.

- 장애물 탐지는 객체탐지와 객체 추적으로 구성됩니다. 먼저 객체 탐지는 라이다로부터 얻은 객체들의 위치와 카메라로부터 얻은 객체를 연관시켜 객체들의 위치, 크기 분류를 합니다. 객체 추적은 위 객체 탐지로부터 얻은 객체들을 칼만 필터 알고리즘을 사용하여 시간대별 객체들의 위치와 속도를 추정합니다. 

- 경로 계획은 전역적 계획과 지역적 계획으로 구성됩니다. 전역적 계획은 벡터 맵과 차량의 pose를 입력받아 기준 경로를 생성합니다. 벡터 맵은 차선, 교통표지판, 신호등 위치등을 벡터 형식으로 표시한 지도입니다. 이 벡터 맵과 포인트 맵을 동기화시켜야 합니다. 지역적 계획은 기준 경로를 입력받아 후보 경로를 생성합니다. 이는 돌발 상황에 대처하기 위함입니다. 그리고 후보경로에서 최종 경로를 선택하여 출력합니다. 

- 경로 추종은 최종 경로를 시간 단위로 나누고 알고리즘에 따라 차량의 타겟 속도와 타겟 조향 각속도를 계산하여 출력합니다. 


In [ ]:
밑에 그림들은 네모는 데이터 값이고 타원은 함수(클래스, 알고리즘)입니다.

In [ ]:
![title](image/12.png)

perception과 detection의 데이터 흐름입니다. image값이 들어와서 이미지 객체와 영역을 표시하고, 라이다 값인 velodyne points값이 들어와서 euclidean cluster을 통해 객체 위치를 탐지합니다. 카메라에와 vector map으로 부터 얻은 객체 label과 라이다로부터 얻은 객체 위치값을 종합하여 객체 정보를 획득합니다.

In [ ]:
![title](image/13.png)

perception과 localization 흐름입니다. 보면 image값이 slam을 만들고 라이다와 GNSS(global navigation Satellite system)을 사용하여 객체를 감지합니다. 그리고 gps로부터 얻은 좌표 데이터를 라이다로부터 얻은 point 데이터를 ndt_matching 알고리즘 을사용하여 tf로 localization을 보냅니다.

![title](image/14.png)

벡터 맵과 각종 waypoints를 가지고 lane_rule을 생성하고 현재 차량의 pose데이터와 합쳐져서 경로를 생성하는 것을 볼 수 있습니다. 그리고 각종 정보를 받아들여 타겟 waypoint를 생성하고 있습니다.

이제 실제 어떻게 구동되는지 확인해 보겠습니다.

![title](image/25.png)

autoware runtime manager 입니다. 
quick start tab을 확인해보면 map, sensing, localization등 각각의 항목에 해당하는 launch 파일을 설정해 줍니다.

![title](image/26.png)

그리고 simulation 칸으로 이동하여 play를 클릭하고 pause를 클릭합니다. 그러면 .bag파일을 읽을 준비가 되었습니다. .bag파일은 ros에서 센서데이터(topic)들을 저장한 파일입니다. 이 센서 데이터는 topics에서 확인 할 수 있습니다. 그리고 topic에서 rviz를 클릭합니다. rviz는 ros의 webviz처럼 visualize 
해주는 툴입니다. 


![title](image/27.png)

이 기능을 활용하기 위해서 map메뉴에서 TF를 활성화 시킵니다. 이 TF는 Frame나 위치 정보를 가지고 있습니다.

그러면 사용할 준비를 확인합니다.

![title](image/28.png)

이제 준비가 끝났습니다. 다시 play버튼을 누르면 .bag파일의 센서데이터들이 재생이 되고 밑의 그림들의 기능이 구현되었음을 확인할 수 있습니다.

![title](image/15.png)

localizaion

![title](image/16.png)

물체 감지

![title](image/17.png)

경로 계획

In [ ]:
![title](image/18.png)

경로추종

In [ ]:
## Yolo

In [ ]:
![title](image/19.png)

yolo는 you only look once의 약자로  실시간 물체 감지 모델입니다 yolo는 현재 v1부터 v5까지 나왔습니다. yolo의 여러 가지 특징 중 가장 좋은 특징은 이미지 전체를 한번만 보는 것입니다. 이전의 모델은 이미지를 여러장으로 분할하고 cnn모델을 활용해 이미지를 분석해서 시간이 오래 걸렸다면 yolo는 이미지를 한번만 보면 이미지를 분석할 수 있습니다. 그리고 두번째 특징은 통합된 모델을 사용한다는 점입니다. 기존의 object detection의 모델은 다양한 모델들을 결합해서 사용했지만 yolo는 통합된 모델을 사용합니다. 그리고 마지막 특성은 실시간 객체 탐지가 가능하다는 점입니다.
yolo의 과정을 v1부터 v3까지 대략적으로 파악하겠습니다. 


먼저 v1에서 이미지를 파악하기 위한 작업 순서는 다음과 같습니다.
- 1. 입력 이미지를 s x s 그리드 영역으로 나누기
- 2. 그리드 영역에서 물체 존재할만한 영역에 2개의 bounding box 예측
- 3. 박스의 신뢰도인 confidence를 계산 (object 확률 *  iou)
- 4. 각각의 그리드마다 c개의 클래스에 대해 해당 클래스일 확률 계산
  - bounding box와 classification 동시수행
- 5. 해당 클래스일 확률과 confidence를 연산하여 box가 특정 클래스일 확률 계산
- 6. NMS알고리즘으로 중복되는 Boxes제거
- 7. FC layer로 최종 연산 


![title](image/20.png)

4번을 수행하기 위해서 초기 yolo v1은 googlenet 아키텍처에서 영감을 얻어 컨볼루션 만으로 네트워크 구성했습니다.  네트워크는224x224크기의 imagenet classification으로 pretrain하고 448x448 크기의 이미지를 입력으로 받았습니다. 그리고 구조는 앞 20개 컨볼루션 레이어는 고정시켜 학습을 못하게 만들고 뒤 4개의 layer만 detection 에 맞게 학습을 시켰습니다. 이때 눈여겨 보아야하는 점은 뒤 layer가 FC layer인 점입니다. 네트워크 출력은 (7x7x30) feature map입니다. 앞의 7x7은 그리드를 의미합니다. 그리고 각각의 인덱스는 30차원의 벡터값을 가지고 있습니다.  30차원중에서 10개는 2개의 bounding box에 해당하고. (x, y, w, h, C) 나머지 20차원은 해당 인덱스가 특정 클래스 일 확률입니다. (클래스가 20개인 데이터 셋)


V2에서는 v1의 여러가지 문제를 해결하기 위해 다양한 방법을 적용합니다.
- v1에서 dropout layer 제거후 batch normalization 적용, 
- 학습전에 image classfication 모델을 큰 해상도 이미지에 대해서 fine-tuning
  - v1의 해상도 문제 해결
- 위에서 7x 7x 30의 피쳐맵을 구함에 있어 앵커박스 개념도입
- v2에서는 passthrough layer을 도입 
  - 상위레이어의 피쳐맵을 하위 피쳐멥에 합침 -> 작은 물체에 대한 정보 소실방지
 - 학습시 여러 스케일의 이미지를 학습할 수 있도록 변경 
 - 기존의 network 대신하여 darknet 고안
   - maxpooling제거, 컨볼루션 연산증가, fc제거
 - 방대한 크기의 class에 대해 계층적 분류작업을 수행 
 - softmax시 대분류로 하기 위해 coco, imageNet등을 트리구조로 변경
 - classification 라벨의 이미지는 classification loss만 역전파 되도록 고안 

=> classification, object detection이 섞인 데이터 셋 학습가능


V3에서는 v2의 모델을 향상시키기 위한 노력을 했습니다. backbone 아키텍처를 darknet-19에서 darknet-53으로 변경하였습니다. 그 이유는 resnet의 skip connection 개념을 도입하여 작은 물체 탐색시 데이터가 유실되는 것을 방지하고 결과적으로 층을 더 많이 쌓기 위함입니다.
darknet-53의 구조의 변경점은 다음과 같습니다. 
- max pooling 대신 컨볼루션 stride를 2로 취해서 해상도를 낮추기.
- res net으로 residual 값을 전달하기.
- average pooling 과 fc를 통과한후 softmax로 출력

이제 v3에 대해 알았으니 v3를 분석해 보겠습니다.
yolo_v3의 darknet/경로로 밑에 파일이 생성이 됩니다.
![title](image/20.png)

backup 파일에는 학습시 weights파일이 생성이 됩니다.
cfg폴더에는 cfg(configuration file) 파일들이 있습니다. cfg파일은 network의 layout을 block단위로 정의한 파일입니다. 내부 파일을 뜯어보면 총 6개의 구조를 지니고 있습니다.



```python
[net]
# Testing
# batch=1
# subdivisions=1
# Training
batch=64
subdivisions=16
width=608
height=608
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1
```

이 block은 네트워크의 전체적인 설정이 포함되어 있습니다. 
- batch : 한번에 처리할 이미지 수입니다.
- subdivisions : mini-batch와 같은 의미 입니다. batch를 분할하는 갯수입니다.
- width : 이미지의 가로 픽셀 수입니다.
- height : 이미지의 세로 픽셀 수입니다.
- channels : 이미지 RGB 3채널을 의미합니다.
- momentum : 경사하강법의 최적화 알고리즘 중 하나입니다. 누적된 과거 그래디언트를 현재 그래디언트에 보정하는 일종의 관성알고리즘입니다. 
- decay : weights updating을 약하게 만듭니다. 
- angle : 학습하는 동안 image를 random하게 회전시킵니다.
- saturation : 학습하는 동안 image의 채도를 random하게 변화시킵니다.
- exposure : 학습하는 동안 image의 노출도를 random하게 변화시킵니다.
- hue : 학습하는 동안 hue(color)를 random하게 변화시킵니다.
- learning_rate : 한번 학습할 때 얼마만큼 학습해야 하는지를 나타내는 학습량입니다.
- burn_in : 몇번의 iteraion이 지날때마다 학습률을 조정 할 것인지에 대한 값.
- max_batches : 얼마나 iteration을 돌지 설정하는 값
- policy : learning rate를 바꾸는 정책, (constant, sgdr, steps, sig, exp, poly, random) 
- steps : max_batches 사이즈의 80,90(%)으로 설정 , 학습되는 단계이다.
- scales = .1,.1



```python
[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky
```
이 블록은 convolutional layer 층의 설정이 포함되어 있습니다.
- batch_normalize : 배치를 다듬는 작업을 합니다. 1이면 사용하고 0이면 사용하지 않습니다.
- fitter : kernel-filter의 갯수입니다.
- size : fitter의 kernel size입니다.
- stride : fitter로 이미지를 스캔할 때의 size입니다. 1이면 한칸씩 움직이고 2이면 Max pooling 과 유사하게 이미지를 다운 샘플링합니다.
- pad : image를 filtering 할때 테두리를 설정합니다. padding = size/2크기로 설정합니다. default는 0입니다.
- activation : 비선형 함수입니다. 추출된 유사도를 더 극적으로 보여주기 위한 함수입니다. leaky는 relu함수의 일종으로 음수일때 완만한 선형 함수, 양수일때 입력값 그대로 return합니다.

```python
[shortcut]
from=-3
activation=linear
```
이 블록은 residual connection (ResNet) 층의 설정이 포함되어 있습니다. ResNet의 기본적인 아이디어는 기존의 학습정보를 보존하고 그 정보를 추가적으로 학습시키는 것입니다. 
- from : 학습정보를 보존할 layer 설정입니다. -3으로 설정하면 shortcut layer뒤의 3번째 layer와 shortcut layer 바로 뒤 layer의 학습정보가 보존이 되어 다음 학습에 사용이 됩니다.
- activation : 위에서 설명한 비선형 함수입니다. linear은 선형함수입니다.  



```python
[yolo]
mask = 6,7,8
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
classes=80
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1
```
이 블록은 yolo의 detection layer입니다. 
- mask는 바로 밑에 있는 anchors의 인덱스입니다.
- anchors는 조정될 bounded_boxes의 initial size입니다.
- classes : detection할 객체의 수입니다.
- num : anchors의 전체 개수 입니다.
- jitter : random하게 crop하고 resize image할 때 비율을 (1- 2*jitter)에서 (1+ 2*jitter)로 변경합니다. 데이터 확대 파라미터는 마지막 층에서만 사용합니다. 
- ignore_thresh : Iou(detect, truth)> ignore_thresh인 경우에만 중복탐지를 합니다. 그리고 NMS(학습하는 동안)을 하는 동안 중복된 것들을 합칠 것입니다. 
- truth_thresh :  Iou(detect, truth)> ignore_thresh인 경우에만 중복탐지 조정을 합니다. 그리고 NMS(학습하는 동안)를 하는 동안 중복된 것들을 합칠 것입니다.
  
      용어 정리 :
- Iou : 두 자료간 교집합 / 두 자료간 합집합 (bounding boxes 기준)
- NMS : 가장 큰 box와 나머지 box간의 Iou값을 계산하여 일정 이상 값을 삭제하는 알고리즘 입니다. 즉 물체가 존재할 가능성이 높은 것을 제외하고 삭제합니다.

```python
[upsample]
stride=2
```
이 블록은 Bilinear upsampling layer입니다. upsampling은 convolution을 통해 데이터를 키우는 방법입니다. 기존 데이터를 stride 거리로 띄워놓고 convolution을 진행하는 방식입니다.
- stride : 데이터를 얼마나 띄워놓을지 결정하는 값입니다.


```python
[route]
layers = -4
```
이 블록은 route layers입니다. route layer는 현재 layer output 와이전 layer output을 연결하는 모듈입니다. 이는 network 초기 부터 미세한 특징을 가져온다는 의미입니다.
- layer : 몇 번째 이전 layer의 output을 가져올지 결정하는 값입니다.



data폴더에는 bounding box를 그릴때 필요한 label, 폰트, test image등이 있습니다.

![title](image/22.png)

내부에 데이터를 보면 9k,coco, inet,openimage, voc의 명칭이 붙은 것을 확인할 수 있습니다. 그리고 9k만 tree파일 이 있는 것을 확인 할 수 있습니다. 이는 yolo가 방대한 크기의 class를 classification하기 위해 계층적으로 분류작업을 하기 위해 만든 tree구조를 사용하는것에 해당되는 파일입니다. coco와 imagenet 등의 데이터셋의 label을 트리구조에 섞어서 만든 것이 9k.tree입니다. 그리고 names에는 각각의 데이터 셋에 해당되는 label (이름)이 적혀있습니다.


example폴더에는 각종 c파일이 있습니다. 이중에서 main file인 darknet.c를 분석하겠습니다.

```c++
#include "darknet.h"
#include <time.h>
#include <stdlib.h>
#include <stdio.h>
```

처음에 darknet.h파일과 필요한 헤더파일을 include합니다.

```c++
extern void predict_classifier(char *datacfg, char *cfgfile, char *weightfile, char *filename, int top);
extern void test_detector(char *datacfg, char *cfgfile, char *weightfile, char *filename, float thresh, float hier_thresh, char *outfile, int fullscreen);
extern void run_yolo(int argc, char **argv);
extern void run_detector(int argc, char **argv);
extern void run_coco(int argc, char **argv);
extern void run_nightmare(int argc, char **argv);
extern void run_classifier(int argc, char **argv);
extern void run_regressor(int argc, char **argv);
extern void run_segmenter(int argc, char **argv);
extern void run_isegmenter(int argc, char **argv);
extern void run_char_rnn(int argc, char **argv);
extern void run_tag(int argc, char **argv);
extern void run_cifar(int argc, char **argv);
extern void run_go(int argc, char **argv);
extern void run_art(int argc, char **argv);
extern void run_super(int argc, char **argv);
extern void run_lsd(int argc, char **argv);
```

각종 필요한 main파일 밖에 있는 함수들을 참조합니다.

그리고 main부분을 먼저 보겠습니다. main안에 들어가는 인자는 예제 명령어 코드를 통해 확인할 수 있습니다. 명령어를 먼저 확인해 보겠습니다. 





``` c++
./darknet detect cfg/yolov3.cfg yolov3.weight data/dog.jpg 
./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weight <video file>
```

명령어는 2가지 종류가 있습니다. 1부터 보면 argv[1]부터 차례대로 detect, cfg파일, weight파일, image파일이 들어갑니다. 그리고  2를 보면 argv[1]부터 차례대로 detector, demo, cfg파일, weights 파일, video 파일이 들어갑니다.

### main 부분

```c++
int main(int argc, char **argv)
{
    //test_resize("data/bad.jpg");
    //test_box();
    //test_convolutional_layer();
    if(argc < 2){
        fprintf(stderr, "usage: %s <function>\n", argv[0]);
        return 0;
    }
    gpu_index = find_int_arg(argc, argv, "-i", 0);
    if(find_arg(argc, argv, "-nogpu")) {
        gpu_index = -1;
    }

#ifndef GPU
    gpu_index = -1;
#else
    if(gpu_index >= 0){
        cuda_set_device(gpu_index);
    }
#endif
```

메인 파일에 입력 파라미터 개수가 2개 미만일때 에러처리를 합니다. 그리고 gpu옵션이 들어와있으면 gpu_index를 할당을 합니다. 만약 -nogpu이면 gpu_index에 -1을 대입합니다. 그리고 make시 gpu옵션이 정의되어 있지 않으면 -1 정의되어 있으면 cuda_set_device함수를 통해 cuda설정을 합니다. 

   ```c++
    if (0 == strcmp(argv[1], "average")){
        average(argc, argv);
    } else if (0 == strcmp(argv[1], "yolo")){
        run_yolo(argc, argv);
    } else if (0 == strcmp(argv[1], "super")){
        run_super(argc, argv);
    } else if (0 == strcmp(argv[1], "lsd")){
        run_lsd(argc, argv);
    } else if (0 == strcmp(argv[1], "detector")){
        run_detector(argc, argv);
    } else if (0 == strcmp(argv[1], "detect")){
        float thresh = find_float_arg(argc, argv, "-thresh", .5);
        char *filename = (argc > 4) ? argv[4]: 0;
        char *outfile = find_char_arg(argc, argv, "-out", 0);
        int fullscreen = find_arg(argc, argv, "-fullscreen");
        test_detector("cfg/coco.data", argv[2], argv[3], filename, thresh, .5, outfile, fullscreen);
    } else if (0 == strcmp(argv[1], "cifar")){
        run_cifar(argc, argv);
    } else if (0 == strcmp(argv[1], "go")){
        run_go(argc, argv);
    } else if (0 == strcmp(argv[1], "rnn")){
        run_char_rnn(argc, argv);
    } else if (0 == strcmp(argv[1], "coco")){
        run_coco(argc, argv);
    } else if (0 == strcmp(argv[1], "classify")){
        predict_classifier("cfg/imagenet1k.data", argv[2], argv[3], argv[4], 5);
    } else if (0 == strcmp(argv[1], "classifier")){
        run_classifier(argc, argv);
    } else if (0 == strcmp(argv[1], "regressor")){
        run_regressor(argc, argv);
    } else if (0 == strcmp(argv[1], "isegmenter")){
        run_isegmenter(argc, argv);
    } else if (0 == strcmp(argv[1], "segmenter")){
        run_segmenter(argc, argv);
    } else if (0 == strcmp(argv[1], "art")){
        run_art(argc, argv);
    } else if (0 == strcmp(argv[1], "tag")){
        run_tag(argc, argv);
    } else if (0 == strcmp(argv[1], "3d")){
        composite_3d(argv[2], argv[3], argv[4], (argc > 5) ? atof(argv[5]) : 0);
    } else if (0 == strcmp(argv[1], "test")){
        test_resize(argv[2]);
    } else if (0 == strcmp(argv[1], "nightmare")){
        run_nightmare(argc, argv);
    } else if (0 == strcmp(argv[1], "rgbgr")){
        rgbgr_net(argv[2], argv[3], argv[4]);
    } else if (0 == strcmp(argv[1], "reset")){
        reset_normalize_net(argv[2], argv[3], argv[4]);
    } else if (0 == strcmp(argv[1], "denormalize")){
        denormalize_net(argv[2], argv[3], argv[4]);
    } else if (0 == strcmp(argv[1], "statistics")){
        statistics_net(argv[2], argv[3]);
    } else if (0 == strcmp(argv[1], "normalize")){
        normalize_net(argv[2], argv[3], argv[4]);
    } else if (0 == strcmp(argv[1], "rescale")){
        rescale_net(argv[2], argv[3], argv[4]);
    } else if (0 == strcmp(argv[1], "ops")){
        operations(argv[2]);
    } else if (0 == strcmp(argv[1], "speed")){
        speed(argv[2], (argc > 3 && argv[3]) ? atoi(argv[3]) : 0);
    } else if (0 == strcmp(argv[1], "oneoff")){
        oneoff(argv[2], argv[3], argv[4]);
    } else if (0 == strcmp(argv[1], "oneoff2")){
        oneoff2(argv[2], argv[3], argv[4], atoi(argv[5]));
    } else if (0 == strcmp(argv[1], "print")){
        print_weights(argv[2], argv[3], atoi(argv[4]));
    } else if (0 == strcmp(argv[1], "partial")){
        partial(argv[2], argv[3], argv[4], atoi(argv[5]));
    } else if (0 == strcmp(argv[1], "average")){
        average(argc, argv);
    } else if (0 == strcmp(argv[1], "visualize")){
        visualize(argv[2], (argc > 3) ? argv[3] : 0);
    } else if (0 == strcmp(argv[1], "mkimg")){
        mkimg(argv[2], argv[3], atoi(argv[4]), atoi(argv[5]), atoi(argv[6]), argv[7]);
    } else if (0 == strcmp(argv[1], "imtest")){
        test_resize(argv[2]);
    } else {
        fprintf(stderr, "Not an option: %s\n", argv[1]);
    }
    return 0;
}
```

argv[1]에 대한 입력으로 어떤 명령을 수행할지 고르는 곳입니다. 이 부분에서 예제에서 사용한 사진에서 객체 찾는 명령어인 detect와 동영상에서 객체 찾는 명령어인 detector을 확인 할 수있습니다. 이 중에서 저는 예제 사진으로 명령어를 돌려보았으므로 detect명령어를 분석하겠습니다

 ```c++
 else if (0 == strcmp(argv[1], "detect")){
        float thresh = find_float_arg(argc, argv, "-thresh", .5);
        char *filename = (argc > 4) ? argv[4]: 0;
        char *outfile = find_char_arg(argc, argv, "-out", 0);
        int fullscreen = find_arg(argc, argv, "-fullscreen");
        test_detector("cfg/coco.data", argv[2], argv[3], filename, thresh, .5, outfile, fullscreen);
    }
```

detect 인수가 argv[1]로 넘어 왔습니다. 이때 각종 명령어를 수행을 합니다. 각각에 대해 살펴보겠습니다.
- find_float_arg를 통해 thresh값이 정의됩니다. 이 함수와 thresh값은 include파일에 darknet.h에 정의되어 있습니다. 
- 파라미터의 개수가 4개 이상이면 argv[4]에 저장된 image file이름을 filename에 저장합니다. 아니면 0을 저장합니다.
- find_char_arg 함수를 통해 outfile이 정의됩니다. 이 함수와 out값은 include파일에 darknet.h에 정의되어 있습니다. 
- find_arg를 함수를 통해 fullscreen이 정의됩니다. 이 함수와 fullscreen값은 include파일에 darknet.h에 정의되어 있습니다. 
- 앞서 만든 모든 파라미터와 coco cfg파일, yolo cfg파일, yolo weight파일을 test_detector함수에 넘겨줍니다.

이제 이미지파일 detect의 핵심인 test_detector함수를 분석하겠습니다.
이 test_detector함수는 example 디렉토리에 detector.c 파일에 있습니다. test_detecot함수는 단일 image에 대해서 객체 탐지와 recognition을 하는 함수입니다.


### <detector.c의 test_detector 부분>

```c++
void test_detector(char *datacfg, char *cfgfile, char *weightfile, char *filename, float thresh, float hier_thresh, char *outfile, int fullscreen)
{
    list *options = read_data_cfg(datacfg);
    char *name_list = option_find_str(options, "names", "data/names.list");
    char **names = get_labels(name_list);

    image **alphabet = load_alphabet();
    network *net = load_network(cfgfile, weightfile, 0);
    set_batch_network(net, 1);
    srand(2222222);
    double time;
    char buff[256];
    char *input = buff;
    float nms=.45;
```

이 함수는 순서대로 cfg/coco.data", argv[2], argv[3], filename, thresh, .5, outfile, fullscreen 파라미터가 넘어옵니다. 
- read_data_cfg(datacfg)는 datacfg의 값을 읽어 options라는 리스트로 받습니다.
- option_find_str(options, "names", "data/names.list")는 위에서 만든 options 리스트 구조체에서 names라는 key값을 찾아 그 값을 name_list 라는 리스트로 받습니다.
- get_labels(name_list)는 위에서 만든 name_list에서 array형태로 label을 때오는 함수입니다.
- load_alphabet()함수는 image 구조체 이중 포인터인 alphabet을 생성합니다. 이 alphabet 안에는 load_image_color()의 return값인 image 정보가 포함되어 있습니다.
load_network(cfgfile, weightfile,0)은 cfg파일과 weightfile을 인자로 받아 network 구조체 net을 반환합니다.
- set_batch_network는 네트워크 구조체와 int값을 인자로 받습니다. network 구조체의 멤버변수 batch의 값은 인자 int값으로 변경합니다.
- srand함수는 rand 함수에 사용될 수를 초기화 합니다.
- 그리고 각종 필요한 변수들을 선언합니다.


```c++
   while(1){
        if(filename){
            strncpy(input, filename, 256);
        } else {
            printf("Enter Image Path: ");
            fflush(stdout);
            input = fgets(input, 256, stdin);
            if(!input) return;
            strtok(input, "\n");
        }
```

이제 반복문을 사용하여 관련 연산을 처리할 것입니다. 우선 if else 문으로 file이 제대로 들어왔는지 체크를 합니다. 제대로 들어와 있으면 input에 filename을 넣어줍니다. 만약 filename이 없으면 image path를 입력하게 합니다.

```c++
 image im = load_image_color(input,0,0);
        image sized = letterbox_image(im, net->w, net->h);
        //image sized = resize_image(im, net->w, net->h);
        //image sized2 = resize_max(im, net->w);
        //image sized = crop_image(sized2, -((net->w - sized2.w)/2), -((net->h - sized2.h)/2), net->w, net->h);
        //resize_network(net, sized.w, sized.h);
        layer l = net->layers[net->n-1];
```

- load_image_color함수가 파일이름을 인자로 받아서 image를 load합니다.
- letterbox_image함수는 image와 net의 입력 이미지 크기은 w, h를 인자로 받습니다. 그리고 image를 정방으로 resize하고 이를 net에 맞게 다시 resize해줍니다.
- net에서 layer중 일부를 저장합니다.


```c++
        float *X = sized.data;
        time=what_time_is_it_now();
        network_predict(net, X);
        printf("%s: Predicted in %f seconds.\n", input, what_time_is_it_now()-time);
```

- resize한 image의 data를 X포인터에 저장합니다.
- 현재 시간을 time변수에 저장합니다.
- net구조체와 image 내부값을 인자로 받아서 forward porp를 합니다. 그 결과를 반환합니다.(src/network.c에 정의되어 있습니다.)



```c++
        int nboxes = 0;
        detection *dets = get_network_boxes(net, im.w, im.h, thresh, hier_thresh, 0, 1, &nboxes);
        //printf("%d\n", nboxes);
        //if (nms) do_nms_obj(boxes, probs, l.w*l.h*l.n, l.classes, nms);
        if (nms) do_nms_sort(dets, nboxes, l.classes, nms);
        draw_detections(im, dets, nboxes, thresh, names, alphabet, l.classes);
        free_detections(dets, nboxes);
        if(outfile){
            save_image(im, outfile);
        }
        else{
            save_image(im, "predictions");
#ifdef OPENCV
            make_window("predictions", 512, 512, 0);
            show_image(im, "predictions", 0);
#endif
        }

        free_image(im);
        free_image(sized);
        if (filename) break;
    }
}
```

get_network_boxes함수는 예측한 bounding boxes의 정보를 return합니다. 그 boxes는 일정 thresh값 미만은 제거된 상태입니다. 그리고 중복되는 boxes를 제거하기 위해서 NMS알고리즘을 사용하였습니다. 그리고 나온 detection된 boxes를 저장하고 save하게 됩니다.

### <src/network.c>

```c++
detection *get_network_boxes(network *net, int w, int h, float thresh, float hier, int *map, int relative, int *num)
{
    detection *dets = make_network_boxes(net, thresh, num);
    fill_network_boxes(net, w, h, thresh, hier, map, relative, dets);
    return dets;
}
```

정의를 살펴보면 net, image의 정보값, thresh값들 그리고 noboxes를 인자로 받습니다. 그 정보를 가지고 make_network_boxes에서 detection(dets) 정보를 생성한 다음 fill_network_boxes에서 filter 상자를 생성합니다.


```c++
detection *make_network_boxes(network *net, float thresh, int *num)
{
    layer l = net->layers[net->n - 1];
    int i;
    int nboxes = num_detections(net, thresh);
    if(num) *num = nboxes;
    detection *dets = calloc(nboxes, sizeof(detection));
    for(i = 0; i < nboxes; ++i){
        dets[i].prob = calloc(l.classes, sizeof(float));
        if(l.coords > 4){
            dets[i].mask = calloc(l.coords-4, sizeof(float));
        }
    }
    return dets;
}
```
num_detections 함수를 통해 얻은 detection정보를 nboxes에 저장한 후 calloc()를 통해 메모리를 할당합니다. 


```c++
int num_detections(network *net, float thresh)
{
    int i;
    int s = 0;
    for(i = 0; i < net->n; ++i){
        layer l = net->layers[i];
        if(l.type == YOLO){
            s += yolo_num_detections(l, thresh);
        }
        if(l.type == DETECTION || l.type == REGION){
            s += l.w*l.h*l.n;
        }
    }
    return s;
}
```

num_detections 함수는  yolo_num_detections 통해 얻은 detection정보를 return 합니다. 함수를 통해 thresh값 미만인 boxes는 정리합니다.    


![title](image/24.jpg)

이 사진은 실제로 yolov3을 구동한 모습입니다.


환경은 ubuntu 18.04, opencv 3.4입니다. 깔기 위해 https://pgmrlsh.tistory.com/4?category=766787 를 참조하였습니다

```c++
git clone https://github.com/pjreddie/darknet
```

명령어로 yolo를 다운받습니다.

```c++
sudo gedit Makefile
```

이제 darknet 디렉토리로 들어가서 Makefile의 내용을 변경합니다, yolo는 bounding box를 사용하기 때문에 opencv설정을 1로 변경합니다.

```python
make
```

darknet 디렉토리로 들어가서 make명령어를 실행합니다. 그러면 yolo 설치와 설정이 끝났습니다.

```python
wget https://pjreddie.com/media/files/yolo.weights
```

위 예제를 사용하기 위해서 yolo에서 제공하는 weight파일을 다운을 받습니다

```python
./darknet detect cfg/yolo.cfg yolo.weights  data/dog.jpg
```
그리고 명령어를 입력합니다. 그러면 위 사진처럼 detection 된 화면을 볼 수 있습니다. 

In [ ]:
https://waymo.com/open/
waymo dataset 사이트

https://www.nuscenes.org/
nuscenes dataset 사이트

https://github.com/TuSimple/tusimple-benchmark/tree/master/doc/lane_detection
Tsimple dataset 사이트

https://webviz.io/
webviz 사이트


https://github.com/Autoware-AI/autoware.ai
autoware 사이트

https://wingnim.tistory.com/56
yolo 사이트
https://yeomko.tistory.com/48?category=888201
yolo 사이트


